In [2]:
import graph_tool.all as gt
import numpy as np
import random
import pandas as pd
import sbm_bipartite

In [3]:
dest="./toy_data.csv"
seed=43
gt.seed_rng(seed) ## seed for graph-tool's random number generator --> same results


In [4]:
model=sbm_bipartite.bipartite_sbm()

In [5]:
model.load_graph(dest)
model.fit()
model.state

<NestedBlockState object, with base <BlockState object with 220 blocks (40 nonempty), degree-corrected, for graph <Graph object, undirected, with 220 vertices and 110 edges, 2 internal vertex properties, 1 internal edge property, at 0x7f609cbbf6e0>, at 0x7f5fd838bb90>, and 6 levels of sizes [(220, 40), (40, 12), (12, 6), (6, 4), (4, 2), (2, 1)] at 0x7f5fd8389400>


<NestedBlockState object, with base <BlockState object with 220 blocks (40 nonempty), degree-corrected, for graph <Graph object, undirected, with 220 vertices and 110 edges, 2 internal vertex properties, 1 internal edge property, at 0x7f609cbbf6e0>, at 0x7f5fd8db3cb0>, and 6 levels of sizes [(220, 40), (40, 12), (12, 6), (6, 4), (4, 2), (2, 1)] at 0x7f60982be120>

In [6]:
size = model.g.new_vp("float")     # creates a VertexPropertyMap of type string
for v in model.g.vertices():
    size[v] = 0.8
    

model.g.vp.size = size

In [7]:
eweight = model.g.new_ep("double")

for e in model.g.edges():
    eweight[e]=0.5

In [21]:
nedges=1000
hide_hierarchy=0
h_v_size=7.0
h_e_size=2.0
file="/home/simone/Downloads/prova6.pdf"

In [23]:
'''
        Plot the graph and group structure.
        optional:
        - filename, str; where to save the plot. if None, will not be saved
        - nedges, int; subsample  to plot (faster, less memory)
        - hide_h, int; wether or not to hide the hierarchy
        - h_v_size, float; size of hierarchical vertices
        - h_e_size, float; size of hierarchical edges
        - **kwargs; keyword arguments passed to self.state.draw method (https://graph-tool.skewed.de/static/doc/draw.html#graph_tool.draw.draw_hierarchy)
        '''
model.state.draw(layout='bipartite', #vertex_text=g.vp.name,# output=filename,
                        vertex_size=size,
                        edge_pen_width=eweight,
                        subsample_edges=nedges, hshortcuts=1, hide=hide_hierarchy,
                        hvprops={'size':h_v_size},
                        heprops={'pen_width':h_e_size},
                        output=file
                       )

(<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7344d6909730, at 0x7344d6990a40>,
 <GraphView object, directed, with 311 vertices and 310 edges, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7344d67fafc0, at 0x7344d67fbb60>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7344d67fafc0, at 0x7344d67fb1a0>, False), at 0x7344d67fafc0>,
 <VertexPropertyMap object with value type 'vector<double>', for Graph 0x7344d67fafc0, at 0x7344db050440>)